In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import pandas_datareader.data as web
import datetime

/Users/danielmpfeffer/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [6]:
# 1984:Q1 to 2015:Q4
start = datetime.datetime(1985, 1, 1)
end = datetime.datetime(2019, 12, 31)

# Quarterly U.S. real GDP from the FRED
ts = web.DataReader(['GDPC1', 'GDPDEF'], 'fred', start, end)

In [8]:
ts = np.log(ts).diff(axis=0)
ts = ts.rename(columns={'GDPC1':'gdp_growth', 'GDPDEF': 'infl_growth'})
ts = ts.dropna()

In [9]:
mod = VAR(ts)
res = mod.fit(maxlags=1)
print(res.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 29, May, 2020
Time:                     13:24:37
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -22.9567
Nobs:                     138.000    HQIC:                  -23.0323
Log likelihood:           1207.17    FPE:                9.43511e-11
AIC:                     -23.0840    Det(Omega_mle):     9.03789e-11
--------------------------------------------------------------------
Results for equation gdp_growth
                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------
const                  0.004913         0.001196            4.107           0.000
L1.gdp_growth          0.371829         0.080094            4.642           0.000
L1.infl_growth        -0.163740         0.192187   

/Users/danielmpfeffer/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  % freq, ValueWarning)


In [10]:
# Eigenvalues of Phi_1
np.linalg.eig(res.coefs)[0]

array([[0.45366449, 0.52986614]])

In [11]:
ts.mean()

gdp_growth     0.006466
infl_growth    0.005306
dtype: float64

In [12]:
res.mean()

array([0.00645084, 0.00525846])

In [13]:
np.cov(ts['gdp_growth'], ts['infl_growth'])

array([[3.17930275e-05, 1.09289459e-06],
       [1.09289459e-06, 5.54773256e-06]])

In [14]:
S_hat = res.acf(0)[0]; S_hat

array([[3.24573646e-05, 1.09283191e-06],
       [1.09283191e-06, 5.67189506e-06]])

In [15]:
from numpy.random import multivariate_normal

def inv_wishart(df, S):
    n = S.shape[0]
    Z = multivariate_normal(np.zeros(n), np.linalg.inv(S), df)
    ZTZ = Z.T @ Z
    return np.linalg.inv(ZTZ)

In [16]:
ts = sm.add_constant(ts)
X = ts.values
XTX_inv = np.linalg.inv(X.T @ X)
Phi_00 = res.intercept[0]
Phi_01 = res.coefs.flatten()[0]
Phi_02 = res.coefs.flatten()[1]
Phi_10 = res.intercept[1]
Phi_11 = res.coefs.flatten()[2]
Phi_12 = res.coefs.flatten()[3]
vec_Phi = [Phi_00, Phi_01, Phi_02, Phi_10, Phi_11, Phi_12]

In [17]:
# Direct sampling
N = 100
post = []
for i in range(N):
    S_iw = inv_wishart(df=137, S=S_hat)
    S_mn = np.kron(S_iw, XTX_inv)
    post.append(multivariate_normal(vec_Phi, S_mn).tolist())

In [18]:
post_df = pd.DataFrame(np.array(post))
post_df = post_df.rename(columns={0:'Phi_00', 1:'Phi_01', 2:'Phi_02', 
                                  3:'Phi_10', 4:'Phi_11', 5:'Phi_12'})

In [20]:
from scipy.stats import bayes_mvs

for i in post_df.columns:
    print(bayes_mvs(post_df[i])[0])

Mean(statistic=0.004895851784024967, minmax=(0.00487768777208448, 0.004914015795965454))
Mean(statistic=0.3723825788217662, minmax=(0.37127050544226176, 0.37349465220127054))
Mean(statistic=-0.161934525931735, minmax=(-0.16489918481281446, -0.15896986705065552))
Mean(statistic=0.001530079714580191, minmax=(0.0015229547524876039, 0.001537204676672778))
Mean(statistic=0.07915427425595717, minmax=(0.07866812364179285, 0.07964042487012149))
Mean(statistic=0.6116748137317916, minmax=(0.6104763455370746, 0.6128732819265085))
